In [1]:
println("Start")

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-AJ0MJTE:4041
SparkContext available as 'sc' (version = 2.4.2, master = local[*], app id = local-1557638333238)
SparkSession available as 'spark'


Start


In [4]:
val ubication = "C:\\Users\\User\\Documents\\education\\spark_definitive_guide\\Spark-The-Definitive-Guide\\data\\flight-data\\csv\\2015-summary.csv"

ubication: String = C:\Users\User\Documents\education\spark_definitive_guide\Spark-The-Definitive-Guide\data\flight-data\csv\2015-summary.csv


In [7]:
val flightData2015 = spark
    .read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv(ubication)

flightData2015: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [8]:
data.show

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [9]:
flightData2015.take(3)

res3: Array[org.apache.spark.sql.Row] = Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344])


In [10]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(2) Sort [count#41 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#41 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#39,ORIGIN_COUNTRY_NAME#40,count#41] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/User/Documents/education/spark_definitive_guide/Spark-The-Defini..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [13]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [16]:
val sqlWay = spark.sql("""
    SELECT DEST_COUNTRY_NAME, count(1)
    FROM flight_data_2015
    GROUP BY DEST_COUNTRY_NAME
""")

sqlWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count(1): bigint]


In [17]:
val dataFrameWay = flightData2015
        .groupBy('DEST_COUNTRY_NAME)
        .count

dataFrameWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count: bigint]


In [18]:
sqlWay.explain

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#39], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#39, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#39], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#39] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/User/Documents/education/spark_definitive_guide/Spark-The-Defini..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [19]:
dataFrameWay.explain

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#39], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#39, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#39], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#39] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/User/Documents/education/spark_definitive_guide/Spark-The-Defini..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [20]:
import org.apache.spark.sql.functions.max
flightData2015.select(max("count")).take(1)

import org.apache.spark.sql.functions.max
res10: Array[org.apache.spark.sql.Row] = Array([370002])


In [21]:
val maxSql=spark.sql("""SELECT DEST_COUNTRY_NAME,sum(count) as destination_total
                        FROM flight_data_2015 
                        GROUP BY DEST_COUNTRY_NAME
                        ORDER BY sum(count) DESC 
                        LIMIT 5""")

maxSql: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, destination_total: bigint]


In [22]:
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [24]:
import org.apache.spark.sql.functions.desc

import org.apache.spark.sql.functions.desc


In [27]:
flightData2015
    .groupBy("DEST_COUNTRY_NAME")
    .sum("count")
    .withColumnRenamed("sum(count)", "destination_total")
    .sort(desc("destination_total"))
    .limit(5)
    .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [28]:
flightData2015
    .groupBy("DEST_COUNTRY_NAME")
    .sum("count")
    .withColumnRenamed("sum(count)", "destination_total")
    .sort(desc("destination_total"))
    .limit(5)
    .explain

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#161L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#39,destination_total#161L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#39], functions=[sum(cast(count#41 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#39, 200)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#39], functions=[partial_sum(cast(count#41 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#39,count#41] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/User/Documents/education/spark_definitive_guide/Spark-The-Defini..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>
